0) Colab runtime + GPU check

In [ ]:
# Check GPU
import torch, platform, sys
print("Python:", sys.version.split()[0])
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
!nvidia-smi

1) System packages (incl. eSpeak dev)

In [ ]:
!sudo apt-get update -y
!sudo apt-get install -y build-essential cmake ninja-build espeak-ng espeak-ng-data libespeak-ng-dev pkg-config ffmpeg
!pkg-config --modversion espeak-ng

2) Clone repo fresh

In [ ]:
%cd /content
!rm -rf piper1-gpl
!git clone https://github.com/OHF-voice/piper1-gpl.git
%cd piper1-gpl
!pwd

3) Python deps (editable install, no venv in Colab)

In [ ]:
!python3 -m pip install --upgrade pip setuptools wheel
!python3 -m pip install -e ".[train]"

4) Build the Cython extension used for alignment

In [ ]:
%cd /content/piper1-gpl
!chmod +x ./build_monotonic_align.sh
!./build_monotonic_align.sh

5) Dev build (repo mode)

In [ ]:
!python3 -m pip install --upgrade pip setuptools wheel scikit-build cmake ninja

In [ ]:
%cd /content/piper1-gpl
!python3 setup.py build_ext --inplace -v

6) (Optional) Mount Google Drive for datasets and outputs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

7) Set paths and training hyperparams

In [ ]:
from pathlib import Path

# ==== CHANGE THESE ====
VOICE_NAME      = "my_colab_voice"
ESPEAK_VOICE    = "en-us"  # run `!espeak-ng --voices` to see options
SAMPLE_RATE_HZ  = 22050
BATCH_SIZE      = 32       # drop to 8 or 4 if you OOM

DATA_ROOT       = Path("/content/drive/MyDrive/tts_data/myvoice")
AUDIO_DIR       = DATA_ROOT / "wavs"
CSV_PATH        = DATA_ROOT / "metadata.csv"

CACHE_DIR       = Path("/content/piper_cache")
CONFIG_PATH     = DATA_ROOT / f"{VOICE_NAME}.json"

# Optional: start from an existing checkpoint to speed up & stabilize training
# Get a .ckpt from https://huggingface.co/datasets/rhasspy/piper-checkpoints (medium quality recommended)
CKPT_PATH       = ""  # e.g., "/content/drive/MyDrive/piper_ckpts/en_US-lessac-medium.ckpt"

# Make sure dirs exist
CACHE_DIR.mkdir(parents=True, exist_ok=True)
print("CSV exists:", CSV_PATH.exists())
print("Audio dir exists:", AUDIO_DIR.exists())
print("Cache dir:", CACHE_DIR)
print("Config will be written to:", CONFIG_PATH)

8) Quick sanity checks

In [ ]:
!espeak-ng --voices | head -n 20

In [ ]:
import pandas as pd, io, os, textwrap

csv_path = str(CSV_PATH)
if os.path.exists(csv_path):
    # Read as pipe-delimited, two columns
    try:
        df = pd.read_csv(csv_path, sep="|", header=None, names=["audio","text"])
        print(df.head())
        # Check a few audio files exist
        missing = [a for a in df["audio"].head(5) if not (AUDIO_DIR/str(a)).exists()]
        print("Missing among first 5:", missing)
    except Exception as e:
        print("CSV read error:", e)
else:
    print("CSV not found at:", csv_path)

9) Kick off training

In [ ]:
!python3 -m piper.train fit \
  --data.voice_name "my_colab_voice" \
  --data.csv_path "/content/drive/MyDrive/tts_data/myvoice/metadata.csv" \
  --data.audio_dir "/content/drive/MyDrive/tts_data/myvoice/wavs" \
  --model.sample_rate 22050 \
  --data.espeak_voice "en-us" \
  --data.cache_dir "/content/piper_cache" \
  --data.config_path "/content/drive/MyDrive/tts_data/myvoice/my_colab_voice.json" \
  --data.batch_size 8 \
  --ckpt_path "https://huggingface.co/datasets/rhasspy/piper-checkpoints/resolve/main/en/en_US/lessac/medium/epoch%3D2164-step%3D1355540.ckpt"

10. Export to onnx

In [ ]:
!python3 -m piper.train.export_onnx \
  --checkpoint "/content/piper1-gpl/lightning_logs/version_2/checkpoints/epoch=2174-step=2680.ckpt" \
  --output-file "/content/model.onnx"

In [ ]:
!cp /content/drive/MyDrive/tts_data/myvoice/my_colab_voice.json /content/model.onnx.json